# Ethical considerations in prompt engineering

As language models become increasingly central to decision-making systems, content generation, and user-facing applications, it's essential that they behave in ways that are not only intelligent but responsible. One critical area in this regard is prompt engineering — the way we phrase our instructions can significantly influence model behavior, and sometimes unintentionally perpetuate harmful biases. The prompts we write shape the responses we get, and careless phrasing can introduce bias, exclude groups, or propagate stereotypes.

This notebook walks through how to detect and mitigate those risks by designing more inclusive, balanced, and fair prompts. We will also introduce simple evaluation techniques to audit model outputs and reflect on their fairness.

In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### Initialize the language model
We instantiate a lightweight GPT model from OpenAI using LangChain.

In [2]:
# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18")

## Understanding biases in AI
Before we can fix bias, we need to see how it shows up. Let's start by examining how biases can manifest in AI responses. We will use a potentially biased prompt and analyze the output.

In [3]:
# A generic prompt that may lead to stereotypical assumptions
biased_prompt = "Describe a typical programmer."

# Call the model with the biased prompt
biased_response = llm.invoke(biased_prompt).content

print("Potentially biased response:")
print(biased_response)

Potentially biased response:
A typical programmer can be characterized by a mix of technical skills, problem-solving abilities, and personal traits. While there is no one-size-fits-all description, here are some common attributes and behaviors associated with programmers:

1. **Technical Skills**: 
   - Proficient in one or more programming languages (e.g., Python, Java, C++, JavaScript).
   - Familiar with software development frameworks and tools (e.g., Git, Docker, IDEs).
   - Knowledge of algorithms, data structures, and design patterns.

2. **Problem-Solving Mindset**: 
   - Enjoys tackling complex challenges and breaking them down into manageable parts.
   - Often employs logical and analytical thinking to debug and optimize code.

3. **Attention to Detail**: 
   - Pays close attention to syntax, semantics, and the overall structure of code.
   - Often reviews and tests code thoroughly to ensure functionality and performance.

4. **Continuous Learner**: 
   - Keeps up with the la

This code sends a prompt with loaded wording—“typical programmer”—which might trigger stereotypical responses. The purpose is to illustrate how implicit bias can surface from vague language.

## Identifying and mitigating biases
Let’s now rewrite the earlier prompt to be explicitly inclusive, and examine the new model output.

In [4]:
# Use a more inclusive prompt to avoid stereotypical assumptions
inclusive_prompt = PromptTemplate(
    input_variables=["profession"],
    template="Describe the diverse range of individuals who work as {profession}, emphasizing the variety in their backgrounds, experiences, and characteristics."
)

# Format and invoke the new prompt
inclusive_response = (inclusive_prompt | llm).invoke({"profession": "computer programmers"}).content
print("More inclusive response:")
print(inclusive_response)

More inclusive response:
The world of computer programming is incredibly diverse, encompassing individuals from a wide range of backgrounds, experiences, and characteristics. Here are some key aspects that highlight this diversity:

1. **Educational Backgrounds**:
   - **Formal Education**: Many programmers hold degrees in computer science, software engineering, or related fields. These individuals often have a solid theoretical foundation in algorithms, data structures, and software design.
   - **Self-Taught**: There are numerous successful programmers who are self-taught, learning through online courses, coding bootcamps, or personal projects. This group often brings unique perspectives and innovative problem-solving skills.
   - **Non-Traditional Paths**: Some programmers come from completely different fields, such as the arts, humanities, or sciences, leveraging their previous experiences to approach programming challenges creatively.

2. **Cultural Diversity**:
   - Programmers h

By reframing the prompt to explicitly ask for diversity, the model is guided to generate a more balanced response that reflects reality rather than stereotypes.

## Creating inclusive prompts across topics
We generalize the technique by writing a helper function to create inclusive prompts for any subject. We will apply it to a few topics prone to cultural or normative bias.

In [5]:
# Function to generate an inclusive prompt for any given topic
def create_inclusive_prompt(topic):
    """Creates an inclusive prompt template for a given topic."""
    return PromptTemplate(
        input_variables=["topic"],
        template="Provide a balanced and inclusive perspective on {topic}, considering diverse viewpoints, experiences, and cultural contexts."
    )

# Test across different social and cultural themes
topics = ["leadership", "family structures", "beauty standards"]

for topic in topics:
    prompt = create_inclusive_prompt(topic)
    response = (prompt | llm).invoke({"topic": topic}).content
    print(f"Inclusive perspective on {topic}:")
    print(response)
    print("\n" + "-"*50 + "\n")

Inclusive perspective on leadership:
Leadership is a multifaceted concept that transcends traditional notions of authority and hierarchy. It is shaped by a variety of factors including cultural backgrounds, personal experiences, and societal contexts. To effectively engage with diverse perspectives on leadership, it is essential to recognize and appreciate the myriad ways in which leadership is understood and practiced across different cultures and communities.

### 1. **Cultural Contexts of Leadership**

Different cultures have distinct values and beliefs that influence their approach to leadership. For instance:

- **Collectivist Cultures**: In many Asian cultures, such as Japan and China, leadership often emphasizes harmony, group cohesion, and consensus-building. Leaders may be seen as facilitators rather than authoritative figures, focusing on the collective rather than individual recognition.

- **Individualistic Cultures**: In contrast, Western cultures, particularly in the Unit

This demonstrates how prompt design can nudge the model to acknowledge complexity and diversity, which is especially critical for topics related to human behavior, culture, or identity.

## Evaluating fairness in AI outputs
After generating model outputs, it’s useful to critically assess them. Let's implement a simple method to evaluate the fairness of AI-generated outputs.

In [6]:
# Function to evaluate fairness and inclusivity in text
def evaluate_fairness(text):
    """Evaluates the fairness of a given text."""
    evaluation_prompt = PromptTemplate(
        input_variables=["text"],
        template="Evaluate the following text for fairness and inclusivity. Identify any potential biases or exclusionary language. Provide a fairness score from 1 to 10, where 10 is most fair and inclusive:\n\nText: {text}\n\nEvaluation:"
    )
    return (evaluation_prompt | llm).invoke({"text": text}).content

# Example text to evaluate
sample_text = "In the corporate world, strong leaders are often characterized by their decisiveness and ability to command respect."
fairness_evaluation = evaluate_fairness(sample_text)

print("Fairness Evaluation:")
print(fairness_evaluation)

Fairness Evaluation:
The provided text describes characteristics of strong leaders in the corporate world, emphasizing decisiveness and the ability to command respect. Here’s an evaluation of its fairness and inclusivity:

1. **Language and Gender Neutrality**: The text uses "their" as a singular pronoun, which is gender-neutral and inclusive. However, the term "strong leaders" might imply a traditional view of leadership that skews towards masculine traits, as "decisiveness" and "commanding respect" can be associated with stereotypical masculine qualities.

2. **Diversity of Leadership Styles**: The text does not recognize that effective leadership can manifest in various styles beyond decisiveness and commanding respect. For example, attributes like empathy, collaboration, and inclusivity are increasingly recognized as vital in modern leadership but are not mentioned here.

3. **Exclusion of Non-Traditional Leaders**: The statement does not acknowledge leaders from diverse background

The model is prompted to score and explain fairness. While not a substitute for human audit, this approach is helpful for rapid iteration or building fairness-checking pipelines.


## Practical exercise
Let’s apply our learnings to improve a biased prompt and compare the model’s behavior before and after the change.

In [7]:
# A prompt that could imply a narrow view of leadership
biased_prompt = "Describe the ideal candidate for a high-stress executive position."

# Show the original response
print("Original prompt:")
print(biased_prompt)
print("\nOriginal response:")
print(llm.invoke(biased_prompt).content)

# An improved, inclusive version of the original prompt
improved_prompt = PromptTemplate(
    input_variables=["position"],
    template="Describe a range of qualities and skills that could make someone successful in a {position}, considering diverse backgrounds, experiences, and leadership styles. Emphasize the importance of work-life balance and mental health."
)

# Show improved prompt and response
print("\nImproved prompt:")
print(improved_prompt.format(position="high-stress executive position"))
print("\nImproved response:")
print((improved_prompt | llm).invoke({"position": "high-stress executive position"}).content)

# Evaluate the fairness of the improved response
fairness_score = evaluate_fairness((improved_prompt | llm).invoke({"position": "high-stress executive position"}).content)

print("\nFairness evaluation of improved response:")
print(fairness_score)

Original prompt:
Describe the ideal candidate for a high-stress executive position.

Original response:
The ideal candidate for a high-stress executive position possesses a unique combination of skills, traits, and experiences that enable them to thrive in demanding environments. Here are the key characteristics:

1. **Resilience**: The ability to bounce back from setbacks, manage stress effectively, and maintain composure under pressure is crucial. Resilient leaders can navigate challenges without losing focus or motivation.

2. **Strong Decision-Making Skills**: The candidate should be adept at making informed, timely decisions, often with limited information. They must weigh risks and benefits quickly and confidently.

3. **Emotional Intelligence**: High emotional intelligence allows the candidate to understand and manage their own emotions, as well as empathize with others. This skill is vital for maintaining team morale and fostering a positive work environment.

4. **Excellent Co

The original prompt implicitly suggests there’s only one "ideal" candidate and may reinforce traditional stereotypes around leadership under stress.

By explicitly encouraging diversity and holistic well-being, the improved prompt elicits a much richer, less stereotyped response that aligns with modern leadership values.

The evaluation gives us a qualitative and quantitative check on the result, helping validate that our rephrased prompt steered the model in a more ethical direction.


Ethical prompt engineering is not just about avoiding harm—it’s about actively shaping model behavior to reflect the values of equity, fairness, and representation. As LLMs become central to communication, search, writing, and decision support, our responsibility as engineers and designers only grows.
